googleで検索して見つけた以下のページの行列の多項式への作用の定義を実装

https://ask.sagemath.org/question/60980/how-to-define-group-actions-on-sagemath/

In [44]:
from sage.categories.action import Action
from sage.groups.matrix_gps.matrix_group import is_MatrixGroup
from sage.rings.polynomial.multi_polynomial_ring import is_MPolynomialRing

class InverseLinearMapSubstitutionAction(Action):
    def __init__(self, G, S):
        if not is_MatrixGroup(G):
            raise TypeError("Not a matrix group: %s" % G)
        if not is_MPolynomialRing(S):
            raise TypeError("Not a multivariate polynomial ring: %s" % S)
        if not G.degree() == S.ngens():
            raise ValueError("Degree of matrix group (%d) does not match number of generators of polynomial ring (%d)" % (G.degree(), S.ngens()))
        if not S.base_ring().has_coerce_map_from(G.base_ring()):
            raise ValueError("Base rings not compatible")
        super().__init__(G, S, is_left=True, op=operator.mul)

    def _act_(self, g, f):
        return f(*(g.inverse() * vector(self.domain(), self.domain().gens())))

    def _repr_name_(self):
        return "inverse linear map substitution action"

剰余類の代表系を計算する関数を実装

In [45]:
from sage.groups.group import is_Group
import time

def cosetRep(G,H):
    # 時間計測開始
    time_sta = time.perf_counter()
    if not is_Group(G):
        raise TypeError("Not a group: %s" % G)
    if not is_Group(H):
        raise TypeError("Not a group: %s" % H)
    
    rep=[]
    g=set(G.list())
    h=set(G(e) for e in H.list())
    coset=set()
    while g:
        p=g.pop()
        rep.append(p)
        coset = set(p*e for e in h)
        g=g-coset
        coset.clear()
        if len(rep)*H.order()==G.order():
            break;
            
    # 時間計測終了
    time_end = time.perf_counter()
    # 経過時間（秒）
    tim = time_end- time_sta

#    print("%d 秒経過" % tim)
    #[結果]
    
    return rep

ルート系とWeyl group $W_G$（の置換群表現版の$S_G$）を生成　※剰余類の代表系計算の軽量化のため、置換群を利用

In [46]:
n=5
Delta_G=RootSystem(['A',n-1])
S_G=WeylGroup(Delta_G,implementation="permutation")
DynkinDiagram(Delta_G)

O---O---O---O
1   2   3   4   
A4

cross outするインデックスを設定

In [47]:
crossed_out_index=[2]
crossed_out_index.sort()

対応するcrossed Dynkin diagramを表示

In [48]:
CartanType(Delta_G).marked_nodes(crossed_out_index).dynkin_diagram()

O---X---O---O
1   2   3   4   
A4 with node 2 marked

simple reflectionを取得

In [49]:
s_G=S_G.simple_reflections()

uncrossed nodeに対応するsimple reflectionを取得

In [50]:
s_H=[ s_G[i] for i in set(range(1,len(s_G)+1))-set(crossed_out_index) ]

uncrossed nodeに対応するsimple reflectionから$W_G$の部分群$W_H$（の置換群表現版の$S_H$）を生成

In [51]:
S_H=S_G.subgroup(s_H)
print('W_Gの位数：%d'%S_G.order())
print('W_Hの位数：%d'%S_H.order())
print('代表系のサイズ：%d'%(S_G.order()/S_H.order()).round())

W_Gの位数：120
W_Hの位数：12
代表系のサイズ：10


冒頭で実装した関数を用いて, $W_G/W_H$（の置換群版の$S_G/S_H$）の代表系を計算

作用させる際は行列としての作用となるため、本来のWeyl group $W_G$を用意し, 代表系を$W_G$の元に変換

In [52]:
W_G = WeylGroup(Delta_G)
W_G_mod_W_H = [W_G(w) for w in cosetRep(S_G,S_H)]

多項式環とWeyl group $W_G$ の多項式環への作用を生成

In [53]:
R = PolynomialRing(QQ, 'x', W_G.degree())
weyl_group_action = InverseLinearMapSubstitutionAction(W_G, R)

変数のリストを用意

In [54]:
x = R.gens(); x

(x0, x1, x2, x3, x4)

positive root $P_G$をambient spaceに用意する.

In [55]:
#ambient spaceを生成
L=Delta_G.ambient_space()

#positive rootをambient spaceの元に変換
P_G=[ L(x) for x in Delta_G.root_lattice().positive_roots() ];P_G

[(1, -1, 0, 0, 0),
 (0, 1, -1, 0, 0),
 (0, 0, 1, -1, 0),
 (0, 0, 0, 1, -1),
 (1, 0, -1, 0, 0),
 (0, 1, 0, -1, 0),
 (0, 0, 1, 0, -1),
 (1, 0, 0, -1, 0),
 (0, 1, 0, 0, -1),
 (1, 0, 0, 0, -1)]

uncrossed nodeから生成されるpositive rootの集合$P_H$をambient spaceに用意する.

In [56]:
# positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
Pi_H=[ x for x in Delta_G.root_lattice().positive_roots()]
for i in crossed_out_index:
    Pi_H=[ x for x in Pi_H if not (x-(Delta_G.root_lattice().simple_roots())[i]).is_positive_root()]
    # ambient spaceの元に変換
P_H=[ L(x) for x in Pi_H ]; P_H

[(1, -1, 0, 0, 0), (0, 0, 1, -1, 0), (0, 0, 0, 1, -1), (0, 0, 1, 0, -1)]

In [57]:
# the weights of the tangenst spaces at torus-fixed points
flag_tangent_weights = set(P_G)-set(P_H)
flag_tangent_weights = [sum( r[l]*x[l] for l in range(len(x))) for r in flag_tangent_weights]

In [58]:
# denominator appearing in localization formula
# (the product of the above weights)
denominator_in_localization = prod(flag_tangent_weights)
# dimension of the flag variety
dim_flag = len(flag_tangent_weights)
t, k, l = var('t k l')
# Todd polynomial (truncated at dim_flag)
td(t) = taylor(t/(1-exp(-t)),t,0,dim_flag)
# truncated exponential function
truncated_exp = lambda x: sum( [x^i /factorial(i) for i in range(dim_flag+1)])

In [59]:
# homogeneous part of degree dim_flag
top_degree_part = lambda F: sum( c*m for c, m in F if m.total_degree() == dim_flag )

In [60]:
# Chern character for the equivariant bundel designated by m times the fundamental weight of the crossed out index
m = 3
ch = truncated_exp(m*sum([x[k] for k in range(crossed_out_index[0])]))
ch_td = (ch*prod([td(l) for l in flag_tangent_weights])).polynomial(QQ)

In [65]:
f = top_degree_part(ch_td)

Numerical computation

In [67]:
random_x = [RealField(1000)(random()) for i in range(W_G.degree())]
orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in W_G_mod_W_H]
integration_numerical = lambda f: sum([f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round()

In [71]:
integration_numerical(f)

175

Comparison with the exact result

In [ ]:
# dimension of the irreducible representation of G
# with m times the fundamental weight of the crossed out index as the highest weight
L.weyl_dimension(m*L.fundamental_weights()[crossed_out_index[0]])

175

Symbolic computation

In [ ]:
sum( [ weyl_group_action(w, f)/weyl_group_action(w, denominator_in_localization) for w in W_G_mod_W_H])

175

Computation of the chern class

In [ ]:
chern = prod(1+x for x in flag_tangent_weights)
degree_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )
chern_degreewise = [ degree_part(chern, i) for i in range(dim_flag + 1) ]

In [75]:
def chern_number(nums: list) -> int: 
    if sum(x for x in nums) != dim_flag:
        0
    else:
        integration_numerical(prod(chern_degreewise[i]^nums[i] for i in range(len(nums))))

0